In [102]:
import bs4.element
from bs4 import BeautifulSoup
import requests
import re

url_base, url_login, url_aggregate, url_result = (
    "https://qalam.nust.edu.pk/",
    "https://qalam.nust.edu.pk/web/login",
    "https://qalam.nust.edu.pk/student/dashboard",
    "https://qalam.nust.edu.pk/student/results/id/"
)
session = requests.Session()
username, password = "madeel.bscs21seecs", "Student.123"
csrf_token = lambda x: x.find("input", {"name": "csrf_token"})["value"]

r = session.get(url_login)
soup = BeautifulSoup(r.text, "html.parser")
csrf_token = csrf_token(soup)
login_data = {"csrf_token": csrf_token, "login": username, "password": password}
r = session.post(url_login, data=login_data)
print(r.status_code == 200)

r = session.get(url_aggregate)
soup = BeautifulSoup(r.content, "html.parser")
data = soup.find_all("div", {
    "class": ["uk-grid", "uk-grid-width-small-1-12", "uk-grid-width-medium-1-12", "uk-grid-width-medium-1-12",
              "uk-grid-width-large-1-4", "uk-margin-medium-bottom"], "data-uk-grid-margin": "",
    "id": "hierarchical-show", "data-show-delay": "100"})[1].find_all("a")

datajson = {}
for i in data:
    datajson[i.find("span", {"class": "md-list-heading md-color-grey-900"}).text] = [
        url_base +
        re.findall(r'''<a data-uk-tooltip="{pos:'top'}" href="(.*?)" title="Open class">''', i.prettify())[0],
        i.find("span", {"class": "md-list-heading md-color-blue-grey-600"}).text,
        i.find("span", {"class": "sub-heading md-color-blue-grey-600"}).text
    ]
print(datajson)

True
{'Database Systems': ['https://qalam.nust.edu.pk//student/results/id/1415485', '4.0', 'CS-220'], 'Computer Organisation & Assembly Language': ['https://qalam.nust.edu.pk//student/results/id/1415486', '4.0', 'CS-235'], 'Data Structures and Algorithms': ['https://qalam.nust.edu.pk//student/results/id/1415487', '4.0', 'CS-250'], 'Technical & Business Writing': ['https://qalam.nust.edu.pk//student/results/id/1415488', '2.0', 'HU-212'], 'Linear Algebra': ['https://qalam.nust.edu.pk//student/results/id/1415489', '3.0', 'MATH-222']}


In [103]:
from concurrent.futures import ThreadPoolExecutor


# with ThreadPoolExecutor() as executor:
#     for key, value in datajson.items():
#         executor.submit(dummy_func, value[0])

In [149]:
r = session.get("https://qalam.nust.edu.pk/student/results/id/1415485")
soup = BeautifulSoup(r.content, "html.parser")

import itertools
import re
import pandas as pds


def extract_data(table, data):
    temp = []
    for i in table.findAll("tr"):
        if i.text not in ['\nAssessment Type\nObtained Percentage\n',
                          '\nAssessment\nMax Mark\nObtained Marks\nClass Average\nPercentage\n']:
            x = i.text
            temp.append(" ".join(x.split()))
    tempVar = ""
    for j in temp:
        if not re.search(r'\d+(\.\d+)?\s+', j):
            tempVar = j
            data[tempVar] = []
        else:
            data[tempVar].append(j.split())

    for ke, val in data.items():
        temp_test = {}
        for kx in val:
            while len(kx) > 5:
                kx[0] += kx[1]
                kx.pop(1)
            temp_test[kx[0]] = {
                "maxMark": kx[1],
                "obtained": kx[2],
                "classAvg": kx[3],
                "percentage": kx[4]
            }
        data[ke] = temp_test

    return data


import bs4

table = soup.findAll("table", {"class": "uk-table uk-table-nowrap uk-table-align-vertical table_tree"})
table = str(table[0]) + str(table[1])
table = BeautifulSoup(table, "html.parser")
data = {}
data = extract_data(table, data)
display(pds.DataFrame(data))

<table class="uk-table uk-table-nowrap uk-table-align-vertical table_tree">
<thead>
<tr>
<th class="uk-width-1-10 md-color-grey-800">Assessment Type</th>
<th class="uk-width-1-10 md-color-grey-800">Obtained Percentage</th>
</tr>
</thead>
<tbody>
<tr class="table-parent-row show_child_row">
<td>
<a class="js-toggle-children-row toggle-childrens" href="#">
                                                                                    Lab Work
                                                                                </a>
</td>
<td class="">
</td>
</tr>
<tr class="table-child-row md-bg-blue-grey-800 md-color-grey-50">
<th>Assessment</th>
<th>Max Mark</th>
<th>Obtained Marks</th>
<th>Class Average</th>
<th>Percentage</th>
</tr>
<tr class="table-child-row">
<td class="uk-width-1-10 md-color-grey-800">
                                                                                        Lab 01
                                                                                    </t

,One Hour Test/Mid Term,Assignment,Quiz,Final,Lab Work
OHT,"{'maxMark': '75.0', 'obtained': '53.75', 'clas...",NaN,NaN,NaN,NaN
Assignment02,NaN,"{'maxMark': '10.0', 'obtained': '9.00', 'class...",NaN,NaN,NaN
Assignment01,NaN,"{'maxMark': '10.0', 'obtained': '7.00', 'class...",NaN,NaN,NaN
Assignment03,NaN,"{'maxMark': '10.0', 'obtained': '7.00', 'class...",NaN,NaN,NaN
Quiz03,NaN,NaN,"{'maxMark': '10.0', 'obtained': '4.00', 'class...",NaN,NaN
Quiz05,NaN,NaN,"{'maxMark': '10.0', 'obtained': '8.00', 'class...",NaN,NaN
Quiz04,NaN,NaN,"{'maxMark': '10.0', 'obtained': '6.00', 'class...",NaN,NaN
Quiz02,NaN,NaN,"{'maxMark': '10.0', 'obtained': '0.00', 'class...",NaN,NaN
Quiz01,NaN,NaN,"{'maxMark': '10.0', 'obtained': '7.00', 'class...",NaN,NaN
Final,NaN,NaN,NaN,"{'maxMark': '100.0', 'obtained': '0.00', 'clas...",NaN


In [105]:
r = session.get("https://qalam.nust.edu.pk/student/results/id/1415486")
soup = BeautifulSoup(r.content, "html.parser")
table = soup.findAll("table", {"class": "uk-table uk-table-nowrap uk-table-align-vertical table_tree"})
print(extract_data(table[0]))
print(extract_data(table[1]))

TypeError: extract_data() missing 1 required positional argument: 'data'

In [151]:
def extract_data(table, data):
    temp = []
    for row in table.find_all('tr'):
        cells = [cell.text.strip() for cell in row.find_all(['th', 'td'])]
        if not any(cell.startswith(('Assessment Type', 'Assessment')) for cell in cells):
            temp.append(cells)
    for cells in temp:
        if not re.search(r'\d+(\.\d+)?', cells[0]):
            key = cells.pop(0)
            data[key] = {}
        else:
            while len(cells) > 5:
                cells[0] += cells[1]
                cells.pop(1)
            data[key][cells[0]] = {
                "maxMark": cells[1],
                "obtained": cells[2],
                "classAvg": cells[3],
                "percentage": cells[4]
            }
    return data


table = soup.findAll("table", {"class": "uk-table uk-table-nowrap uk-table-align-vertical table_tree"})
table = str(table[0]) + str(table[1])
table = BeautifulSoup(table, "html.parser")
data = {}
data = extract_data(table, data)
display(pds.DataFrame(data))

,One Hour Test/Mid Term,OHT,Assignment,Quiz,Final,Lab Work
Assignment 02,NaN,NaN,"{'maxMark': '10.0', 'obtained': '9.00', 'class...",NaN,NaN,NaN
Assignment 01,NaN,NaN,"{'maxMark': '10.0', 'obtained': '7.00', 'class...",NaN,NaN,NaN
Assignment 03,NaN,NaN,"{'maxMark': '10.0', 'obtained': '7.00', 'class...",NaN,NaN,NaN
Quiz03,NaN,NaN,NaN,"{'maxMark': '10.0', 'obtained': '4.00', 'class...",NaN,NaN
Quiz05,NaN,NaN,NaN,"{'maxMark': '10.0', 'obtained': '8.00', 'class...",NaN,NaN
Quiz04,NaN,NaN,NaN,"{'maxMark': '10.0', 'obtained': '6.00', 'class...",NaN,NaN
Quiz02,NaN,NaN,NaN,"{'maxMark': '10.0', 'obtained': '0.00', 'class...",NaN,NaN
Quiz01,NaN,NaN,NaN,"{'maxMark': '10.0', 'obtained': '7.00', 'class...",NaN,NaN
Lab 01,NaN,NaN,NaN,NaN,NaN,"{'maxMark': '10.0', 'obtained': '10.00', 'clas..."
Lab 06,NaN,NaN,NaN,NaN,NaN,"{'maxMark': '10.0', 'obtained': '10.00', 'clas..."
